# Remote

In [11]:
# Set working directory
domain = "fend01"
dir = "exp/prota_h1-0"
partition = "sbinlab_gpu"

remotedir = f"~/OrthoIDP/{dir}"

print(f"CONTENTS IN {remotedir} AT {domain.upper()}:")
!ssh $domain "cd $remotedir && ls"

print(f"\nCONTENTS IN {remotedir}/results AT {domain.upper()}:")
!ssh $domain "cd $remotedir/results && ls"

CONTENTS IN ~/OrthoIDP/exp/prota_h1-0 AT FEND01:
data
prota_h1-0.ipynb
prota_h1-0.json
results
submit_single_chain.sh

CONTENTS IN ~/OrthoIDP/exp/prota_h1-0/results AT FEND01:
single_chain
single_chain.err
single_chain.out


In [2]:
# Check availability
!ssh $domain sinfo -p $partition

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
PARTITION   AVAIL  TIMELIMIT  NODES  STATE NODELIST
sbinlab_gpu    up   infinite      1 drain* node152
sbinlab_gpu    up   infinite      2    mix node[150-151]


In [11]:
# Check queue
!ssh $domain "squeue -u fknudsen"

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          38863362 sbinlab_g h1-0_min fknudsen  R 6-08:02:54      1 node150
          38863363 sbinlab_g h1-0_max fknudsen  R 6-08:02:54      1 node150
          38868112 sbinlab_g h1-0_con fknudsen  R 6-02:27:13      1 node150


In [27]:
# Check submit script
!ssh $domain "cat $remotedir/submit.sh"

#!/bin/bash
#SBATCH --job-name=initial
#SBATCH --partition=sbinlab_gpu
#SBATCH --array=0-5%6
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --gres=gpu:1
#SBATCH -t 24:00:00
#SBATCH -o results/out
#SBATCH -e results/err


# Getting job input file
input_files=($(ls data/*.fasta))
input_file=${input_files[$SLURM_ARRAY_TASK_ID]}

# Displaying job info
echo "[`date`] STARTED Job Array ID: $SLURM_ARRAY_TASK_ID | Job ID: $SLURM_JOB_ID | Input: $input_file"

# DeiC env settings
source /groups/sbinlab/fpesce/.bashrc
conda activate openmm

# Submitting simulation
python ../../src/simulate_openmm.py -f $input_file

echo "[`date`] FINISHED Job Array ID: $SLURM_ARRAY_TASK_ID | Job ID: $SLURM_JOB_ID | Input: $input_file"


In [13]:
# Submit
!ssh $domain "cd $remotedir && sbatch submit.sh"

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
/Users/frederikespersenknudsen/Local/OrthoIDP


In [ ]:
# Cancel job
jobid = ""
!ssh $domain scancel $jobid

In [12]:
# Copy results to BINF
!scp -r "$domain:$remotedir/results" $dir

restart.chk                                   100%   87KB   2.6MB/s   00:00    
system.xml                                    100%   27KB   1.1MB/s   00:00    
simulate.log                                  100%  652    39.6KB/s   00:00    
traj.log                                      100% 2376KB   7.5MB/s   00:00    
traj.dcd                                      100%   44MB  10.1MB/s   00:04    
top.pdb                                       100%   11KB 507.9KB/s   00:00    
traj.dcd                                      100%   13MB   9.2MB/s   00:01    
system.xml                                    100% 8279   732.2KB/s   00:00    
top.pdb                                       100% 3141   128.2KB/s   00:00    
traj.log                                      100% 2375KB   5.2MB/s   00:00    
restart.chk                                   100%   76KB   2.4MB/s   00:00    
simulate.log                                  100%  567    49.6KB/s   00:00    
simulate.log                            

In [6]:
!ls $dir/results

0  10  4.258203


In [9]:
# Copy single specific files to BINF
filename = "4.258203/evolution.pkl"
!scp "$domain:$remotedir/results/$filename" "$dir/results/$filename"

evolution.pkl                                 100%  419KB   6.0MB/s   00:00    
